In [1]:
from bertopic import BERTopic
from nltk.corpus import stopwords
from functions import *
from glob import glob as gb
from conllu import parse
import pandas as pd
import numpy as np
import spacy
import os 

stopwords_ = {}
stopwords_.update({'gb':stopwords.words('english') + "hon honorable government parliament gentleman gentlemen friend right minister".split(' ')})
stopwords_.update({'it':stopwords.words('italian') + "ministro ministra signore signori parlamento comitato governo presidente presidentessa segretario segretaria".split(' ')})
stopwords_.update({'nl':stopwords.words('dutch') + "minister parlement wij zij het motie moeten komen debat voorzitter nederland".split(' ')})

data_path = "/media/ruben/Elements/ParlaMint/data"
data_transformed_path = "/media/ruben/Elements/ParlaMint/data_transformed"


# tf = pd.read_csv(base_path + '/resources/keywords-expertise-translation.csv')
# tf_science = {language:dict(zip(tf[tf['language'] == language]['word'],tf[tf['language'] == language]['translation'])) for language in list(set(tf['language']))}
# tf_science = {k:{x:i for x,i in v.items() if x not in ["policy","program","measures"]} for k,v in tf_science.items()}
# tf = pd.read_csv(base_path + '/resources/keywords-corona-translation.csv')
# tf_corona = {language:dict(zip(tf[tf['language'] == language]['word'],tf[tf['language'] == language]['translation'])) for language in list(set(tf['language']))}

In [2]:
# Load and Clean

def train_model(iso):
    df = data_loader.load_month(iso,"2020-03","2020-08")
    df['text'] = [utils.preprocess(str(x),stopwords_=stopwords_[iso]) for x in df['lemmatized']]

    if len(df) > 10000:
        df = df.sample(10000).reset_index(drop=True)
    docs = list(df['text'])
    if iso == 'gb':
        topic_model = BERTopic(language="english",n_gram_range=(1,2),min_topic_size=10, calculate_probabilities=True, verbose=True)
        topics, probs = topic_model.fit_transform(docs)
    else:
        topic_model = BERTopic(language="multilingual",n_gram_range=(1,2),min_topic_size=10, calculate_probabilities=True, verbose=True)
        topics, probs = topic_model.fit_transform(docs)

    df['topic'] = topics
    df_prob = pd.DataFrame(probs)

    try:
        topic_model.save(f"bertopic-{iso}")
    except Exception as e:
        print(e)
    df.to_csv(f'/home/ruben/Documents/GitHub/ParlaMintCase/results/models/bertopic-{iso}-date.csv',sep='\t')	
    df_prob.to_csv(f'/home/ruben/Documents/GitHub/ParlaMintCase/results/models/bertopic-{iso}-probs.csv',sep='\t')	

In [3]:
for iso in ['gb','nl','it']:
    print(iso)
    train_model(iso)

gb
Batches: 100%|██████████| 313/313 [08:54<00:00,  1.71s/it]
2021-06-02 21:34:53,507 - BERTopic - Transformed documents to Embeddings
2021-06-02 21:35:19,866 - BERTopic - Reduced dimensionality with UMAP
2021-06-02 21:35:26,899 - BERTopic - Clustered UMAP embeddings with HDBSCAN
/home/ruben/.local/lib/python3.8/site-packages/scipy/sparse/_index.py:124: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


AttributeError: 'function' object has no attribute 'im_func'

In [37]:
tm.save('my_model')

In [22]:
topic_model = BERTopic.load('/home/ruben/Documents/GitHub/ParlaMintCase/results/models/bertopic-gb')

In [28]:
texts = pd.read_csv(f'/home/ruben/Documents/GitHub/ParlaMintCase/results/models/bertopic-gb.csv',sep='\t')	
texts = [str(x) for x in list(texts['text'])]

In [30]:
topic_model.get_topic_info()

ValueError: This BERTopic instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [31]:
# Plot 

topwords = dict(zip(freq['Topic'],freq['Name']))
tdf = pd.DataFrame(topics,columns=['topic'])
tdf['n'] = 1
tdf['week'] = df['week']
tdf = tdf.groupby(['topic','week']).sum().reset_index().pivot(index='week',columns='topic',values='n')
tdf = tdf.T

for c in tdf.columns:
    tdf[c] = tdf[c] / tdf[c].sum()
tdf = tdf.T.reset_index()
tdf.plot(x='week',y=list(tdf.columns)[1:],figsize=(20,10),cmap='tab20b',marker = 'o',legend=sorted(list(freq['Name'])))